## PhidgetSlider_v3 notebook

November 19. M. Lamoureux

Test the slider Phidget attached to the VINT device.

#### Setup

- plug in the Phidget VINT Hub to the computer's USB port
- Plug in the Phidget slider device (part number POT2001) to Port 1 on the VINT
- (You can use a different port, but you need to change the code below.)
- The code below creates a phidgetSlider object in Python, sets the port number to 1, then displays the phidget widget
- click on the button on the screen to connect the phidget
- you can see the values displayed. Slide the slider to change values
- additional code links the physical slider with an ipywidget slider on screen

Follows the code model from the Phidgets company, called VoltageRatioInput.html


## Pip install

The pip install only needs to be done once on your hub. I include it here for completeness.

In [2]:
!pip install "anywidget[dev]"

  Using cached anywidget-0.7.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached psygnal-0.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached watchfiles-0.21.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
Using cached psygnal-0.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (614 kB)
Using cached watchfiles-0.21.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
Using cached anywidget-0.7.1-py3-none-any.whl (40 kB)


In [9]:
import anywidget
import traitlets

In [10]:
class phidgetSlider(anywidget.AnyWidget):
    _esm = """
import  "https://unpkg.com/phidget22/browser/phidget22";

export function render({ model, el }) {
    var conn = 0;
    var ch = 0;
    var isOpen = false;
    var connOpen = false;
    var chOpen = false;

    async function openUSB() {
        if (conn) {
            try {await ch.close();} catch {console.log('ch.close failed');}
            try {await conn.close();} catch {console.log('conn.close failed');}
            try {await conn.delete();} catch {console.log('conn.delete failed');}
        }
        conn = new phidget22.USBConnection();
        ch = new phidget22.VoltageRatioInput();
        conn.onError = function(code, msg) {
            console.log('Conn Code: ' + code);
            console.log('Conn error: ' + msg);
            console.log('----------');
        };
        ch.onError = (code, msg) => {
        	console.log('Ch Code: ' + code);
        	console.log('Ch error: ' + msg);
        	console.log('----------');
        };
        ch.onAttach = onAttach;
        ch.onDetach = () => {
        	console.log('onDetach called. ');
        }
        ch.setIsHubPortDevice(true);
        ch.setHubPort(model.get("hubPort"));
    	try {
    		await conn.connect();
            conn.requestWebUSBDeviceAccess();
            console.log('usb connected');
            connOpen = true;
    	} catch(err) {
    		console.log("Conn open error: ", err);
            connOpen = false;
    	}
        if (connOpen) {
            try {
                await ch.open();
                console.log('Channel connected');
                chOpen = true;            
            } catch(err) {
        		console.log("Channel open error: ", err);
                chOpen = false;
            }
        }
        // now get the UI to reflect the status
        if (connOpen && chOpen) {
            isOpen = true;
            textStatus.innerHTML = 'USB connected, channel open.';
            button.innerHTML = `Click to disconnect`;
        }
        if (connOpen && !chOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB connected, channel not open.';
            button.innerHTML = `Click to connect`;
        }
        if (!connOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB did not connect.';
            button.innerHTML = `Click to connect`;        
        }
    };

    async function closeUSB() {
        console.log('closeUSB called');
        try {await ch.close();} catch {console.log('ch.close failed');}
        try {await conn.close();} catch {console.log('conn.close failed');}
        try {await conn.delete();} catch {console.log('conn.delete failed');}
        isOpen = false; 
        connOpen = false;
        chOpen = false;
        textStatus.innerHTML = 'Disconnected.  ';
        textValue.innerHTML = 'Value is null.';
        button.innerHTML = `Click to connect`;
    };
    model.on('msg:custom', msg => {
        console.log(`new msg received: ` + msg);
        if (msg == 'open') {
            console.log('open msg called');
            openUSB();
         }
         if (msg == 'close') {
            console.log('close msg called');
            closeUSB();
         }
    });
    function onAttach(ch) {
        console.log('channel is attached');
        textStatus.innerHTML = 'Attached.  ';
    	ch.onVoltageRatioChange = ratioChange;
        ch.setDataInterval(100);
        ch.setVoltageRatioChangeTrigger(.01);
    }
    function ratioChange(ratio) {
        textValue.innerHTML = 'Value is ' + ratio;
        model.set('value', ratio);
        model.save_changes();
    }
    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to connect`;
    button.addEventListener("click", async () => {
        if (isOpen) {closeUSB();} else {openUSB();}
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Message area.';
    let textValue = document.createElement("label");
    textValue.innerHTML = 'Value area.';
    let div0 = document.createElement("div");
    el.appendChild(button);
    el.appendChild(textStatus);
    div0.appendChild(textValue);
    el.appendChild(div0);
    // we include a return function to close the Phidget when the notebook is closed
    return async () => {
        try {await ch.close();} catch {}
        try {await conn.close();} catch {}
        try {await conn.delete();} catch {}
    };
    }
    """
    _css="""
    .ph-button {
        color: white; 
        background-color: var(--md-blue-700); 
        border-radius: 8px; 
        font-size: 24px; 
        display: block;
        padding: 15px 32px;
    }
    .ph-button:hover { 
        background-color: var(--md-blue-900); 
    }
    """
    hubPort = traitlets.Int(0).tag(sync=True)
    value = traitlets.Float(0).tag(sync=True)

ps = phidgetSlider()
ps.hubPort = 1


In [11]:
ps

phidgetSlider(hubPort=1)

In [12]:
from ipywidgets import FloatSlider

In [13]:
mySlider = FloatSlider(
    min=0,max=1.0,step=0.01,
    description='Value',readout_format='.3f',
)

def updateSlider(change):
    mySlider.value = change.new

ps.observe(updateSlider, names=["value"])

mySlider

FloatSlider(value=0.0, description='Value', max=1.0, readout_format='.3f', step=0.01)

In [7]:
# Here are some useful commands to access the widget in code

#ps.send("open")     ## open the phidget
#ps.value            ## read the value
#ps.send("close")    ## close the phidget

In [14]:
ps.value

0.43356